In [1]:
import pandas as pd
import git
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# assuming data is in 
repo = git.Repo('.', search_parent_directories=True)
path = Path(repo.working_tree_dir).joinpath('data')
assert path.exists
files = list(path.glob("*.txt"))

In [3]:
dfs = {}
for file in files:    
    df = pd.read_csv(file, index_col=0)
    df = df.rename_axis(index="time")
    df["DOTm"] /= 100
    df.name = "".join([c for c in file.name if c.isdigit()])
    dfs[df.name] = df

df

In [13]:
plt.style.use('ggplot')
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(16, 8), sharex=True, sharey=True, tight_layout=True, subplot_kw={'yscale':'log'})

for df, ax in zip(dfs.values(), axes.flatten()):
    x = df.index
    for col in df.columns:
        if col in  ('Biomass', 'Substrate', 'Acetate', 'Product'):
            y=df[col]
            mask = pd.notna(y)
            line, = ax.plot(x[mask], y[mask], ls="--", lw=1, marker="*")
            ax.plot(x,y, color=line.get_color(), lw=1.5)
            ax.set_title(df.name)

In [5]:
import os
if os.path.exists('in_silico_data.h5'):
    os.remove('in_silico_data.h5')
    
for df in dfs.values():
    df.to_hdf('in_silico_data.h5', key=df.name, mode='a')

In [6]:
import h5py
import pandas as pd

with h5py.File('in_silico_data.h5', "r") as file:
    read_dfs = {}
    for key in file.keys():
        read_dfs[key] = pd.read_hdf('in_silico_data.h5', key=key)

In [7]:
for key in dfs:
    pd.testing.assert_frame_equal(dfs[key], read_dfs[key])

In [11]:
read_dfs['16130']